In [3]:
"""
how to checkout pygt3x module for non-local run?
pip install gitpython
import git
from git import Repo  # pip install gitpython

Repo.clone_from("https://github.com/actigraph/pygt3x.git", working_dir)"""

import calendar
from datetime import datetime, timedelta
import os
import numpy as np
import pandas as pd
import sys
sys.path.append('/Users/adam.santorelli/lib/pygt3x/')
import pygt3x
from pygt3x.reader import FileReader
from typing import Any
import pydantic
import rdata
from matplotlib import pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots


ModuleNotFoundError: No module named 'pygt3x.reader'

In [ ]:
#user defined path to .gt3x data
#currently assumes only one file

data_dir = "/Users/adam.santorelli/Documents/"
for file in os.listdir(data_dir):
    if file.endswith(".gt3x"):
                file_name = os.path.join(data_dir, file)
file_name

In [ ]:
with FileReader(file_name) as reader:
    df = reader.to_pandas()

In [ ]:
timestamps = df.index.to_numpy()
time_fix = []
for i in range(len(timestamps)):
    tmp = (timestamps[0] + ((i)/30))*1000
    time_fix.append(tmp)




In [ ]:

time_fix_test = np.asarray(time_fix, dtype='datetime64[ms]')
del(time_fix)

In [ ]:
ENMO_calc = np.asarray(np.sqrt(np.square(df).sum(axis =1))-1)
angle_z_raw = np.asarray(np.arctan(df.Z / (np.sqrt( np.square(df.X) + np.square(df.Y)))) / (np.pi/180))
data_raw = {'time': time_fix_test, 'ENMO': ENMO_calc, 'angle_z': angle_z_raw}

df_raw = pd.DataFrame(data_raw)
del(data_raw, ENMO_calc, timestamps, time_fix_test, angle_z_raw, df)


In [ ]:
#get day and month values
df_raw['month'] = df_raw['time'].dt.month
df_raw['day'] = df_raw['time'].dt.day


In [ ]:
# get day idx relative to jan 1 ...issue if the participant has data across calendar year...
day_idx = []
for k in range(len(df_raw)):
    tmp = datetime.strptime(str(df_raw['time'][k])[0:10], '%Y-%m-%d')
    tmp = tmp.timetuple().tm_yday
    day_idx.append(tmp)

df_raw['day_idx'] = day_idx

In [ ]:
#number of unique days
days_raw = pd.unique(df_raw.day_idx)



In [ ]:
#plotting plans, request number of days to compare across initial compare for raw data
"""day1 = df_raw['day_idx'][0]
num_days = 2


fig = make_subplots(rows = num_days, cols= 1)
fig2 = make_subplots(rows = num_days, cols= 1)
for k in range(num_days):
    tmp = df_raw[df_raw['day_idx'] == (day1+k)]
    fig.add_trace (go.Scatter(x=tmp['time'], y = tmp['ENMO']), row=k+1, col=1)
    fig2.add_trace (go.Scatter(x=tmp['time'], y = tmp['angle_z']), row=k+1, col=1)
fig.show()
fig2.show()"""

In [ ]:
# switch to R data
# assumes GGIR output in /meta/basic/ ... can prompt for output dir from user

ggir_output_dir =  data_dir + "/output_NDARZN578YDP/meta/basic/"
for file in os.listdir(ggir_output_dir):
    if file.endswith(".RData"):
                Rdata_file_name = os.path.join(data_dir, ggir_output_dir, file)
Rdata_file_name

In [ ]:
# load GGIR ouptut R data and convert to df

R_data = rdata.parser.parse_file(Rdata_file_name)
R_data_conv = rdata.conversion.convert(R_data) 

In [ ]:
#grab key metrics computed by GGIR
ENMO_R_data = R_data_conv['M']['metashort']['ENMO']
z_angle_R_data = R_data_conv['M']['metashort']['anglez']
time_R = R_data_conv['M']['metashort']['timestamp']

#strip out tz info to match raw ...issues with changing tz...
time_R = time_R.str[:-5]

In [ ]:
dti = pd.to_datetime(time_R)
data_R = {'time': dti, 'ENMO': ENMO_R_data, 'anglez':z_angle_R_data}
df_R = pd.DataFrame(data_R)
df_R['month'] = df_R['time'].dt.month
df_R['day'] = df_R['time'].dt.day


del(data_R, time_R, ENMO_R_data, z_angle_R_data, R_data, dti)

In [ ]:
#get day idx as with raw
day_idx_R = []
for k in range(len(df_R)):
    tmp = datetime.strptime(str(df_R['time'][k+1])[0:10], '%Y-%m-%d')
    tmp = tmp.timetuple().tm_yday
    day_idx_R.append(tmp)

df_R['day_idx'] = day_idx_R

In [ ]:
#plotting plans, request number of days to compare across


#usr defined number of days
num_days = 3

#usr defined start date
start_date = 3

#indexing is unique for reasons...
day1_raw = df_raw['day_idx'][0]
day1_R = df_R['day_idx'][1]

day_start_raw = start_date + day1_raw
day_start_R = start_date + day1_R

In [ ]:

#each day will be a separate row
fig = make_subplots(rows = num_days, cols= 1)
fig2 = make_subplots(rows = num_days, cols= 1)

#pass subplot title as for i in range(num_days): title_days[i] = 'day' + str(num_days(k))?? 

for k in range(num_days):
    tmp_raw = df_raw[df_raw['day_idx'] == (day_start_raw+k)]
    tmp_r = df_R[df_R['day_idx'] == (day_start_R+k)]

    fig.add_trace (go.Scatter(x=tmp_raw['time'], y = tmp_raw['ENMO'], name = "Raw"), row=k+1, col=1)
    fig.add_trace (go.Scatter(x=tmp_r['time'], y = tmp_r['ENMO'], name = "GGIR output"), row=k+1, col=1)

    fig2.add_trace (go.Scatter(x=tmp_raw['time'], y = tmp_raw['angle_z'], name = "Raw"), row=k+1, col=1)
    fig2.add_trace (go.Scatter(x=tmp_r['time'], y = tmp_r['anglez'], name = "GGIR output"), row=k+1, col=1)


fig.update_layout(title="ENMO Raw vs GGIR", yaxis_title = 'ENMO')
fig2.update_layout(title = "Angle z Raw vs GGIR", yaxis_title = "Angle Z")

In [22]:
fig.write_html(data_dir + str(num_days) + "days_enmo_compare.html")
fig2.write_html(data_dir + str(num_days) + "days_angle_z_compare.html")

In [ ]:
fig.show()
fig2.show()